In [37]:
import pandas as pd
import numpy as np
import h5py
import os
from pandas import HDFStore
from datetime import datetime,timedelta,date
from dataclasses import dataclass
from typing import Any
from collections import defaultdict

In [38]:
def get_n_days_ago(n=0,time_format="%d-%m-%Y"):
    time_stamp = datetime.now()-timedelta(days=n)
    return time_stamp.strftime(time_format)

In [57]:
datetime.now().strftime("%Y%m%d_%H%M%S")

'20210923_112539'

In [2]:
def get_mem_usage(df):
    print(f"{df.memory_usage(deep=True).sum()/1024 **2:3.2f}Mb")

In [3]:
os.chdir('C:\\Users\\attra')

In [4]:
employe = pd.read_excel('employee.xlsx')
employe

,emp_id,first_name,last_name,hire_date
0,100,Steven,King,2003-06-17
1,101,Neena,Kochhar,2005-09-21
2,102,Lex,De Haan,2001-01-13
3,103,Alexander,Hunold,2006-01-03
4,104,Bruce,Ernst,2007-05-21
5,105,David,Austin,2005-06-25
6,106,Valli,Pataballa,2006-02-05
7,107,Diana,Lorentz,2007-02-07
8,108,Nancy,Greenberg,2002-08-17
9,109,Daniel,Faviet,2002-08-16


In [5]:
numbers = pd.DataFrame(np.random.rand(10_000_000,4),columns=['col_1','col_2','col_3','col_4'])

In [6]:
import os
os.chdir('F:\\Data')
os.getcwd()

'F:\\Data'

In [85]:

class MyHDF5:
    
    current_path = os.getcwd()
    cursor =  Any
    zip_level = 0
    hdf5_name = ""
    def __init__(self,name:str,path:str = None,compressed_level:int = 0) -> pd.HDFStore:
        
        self.hdf5_name = f"{name}.h5"
        if compressed_level:
            self.zip_level = compressed_level
        if path:
            os.chdir(path)
            self.cursor = pd.HDFStore(self.hdf5_name,complevel=self.zip_level)
            os.chdir(self.current_path)
        self.cursor = pd.HDFStore(self.hdf5_name,complevel=self.zip_level)
        print(self.cursor.info())       
        
    def insert_df(self,key,df,timestamp=False):
        data_location = key
        if timestamp==True:
            time_label = datetime.now().strftime("%Y%m%d_%H%M%S")
#             time_label = datetime.now().strftime("%Y%m%d")
            data_location=f"{key}/{time_label}"
            
        try:
            print("Inserting...")
            self.cursor.put(key=data_location,value=df)
            print(f"Insert df success, key: <<{data_location}>>, dataset size: {get_mem_usage(df)}")
        except Exception as e:
            pass
#             print(e)

            
    def extract_data(self,key):
        return self.cursor[key]
    
    def remove_data(self,key):
        try:
            del self.cursor[key]
            print(f"Remove dataset {key} success")
        except KeyError:
            raise KeyError(f"{key} not found")
    def append_df(self,key,df:pd.DataFrame):
        self.cursor[key] = pd.concat([self.cursor[key],df])
        
        
    
    def __repr__(self)->None:
        return(self.cursor.info())
    
    def is_open(self)->bool:
        return self.cursor.is_open
    
    @property
    def keys(self)->list:
        return sorted(list(self.cursor.keys()))
    
    def __getitem__(self,key:str)->"dataset":
        return self.cursor[key]
        
    def __setitem__(self,key,value)->None:
        self.cursor[key]=value
    
    def __delitem__(self,key)->None:
        del self.cursor[key]
    
    def __contains__(self, key: str) -> bool:
        """
        check for existence of this key
        can match the exact pathname or the pathnm w/o the leading '/'
        """
        node = self.cursor.get_node(key)
        if node is not None:
            name = node._v_pathname
            if name == key or name[1:] == key:
                return True
        return False
    
    def __len__(self) -> int:
        return len(self.cursor.groups())
    
    def items(self):
        """
        iterate on key->group
        """
        for g in self.cursor.groups():
            yield g._v_pathname, g
            
    def close(self)->None:
        try:
            self.cursor.close()
            print("Close success")
        except Exception as e:
            print(e)
            
    def check_groups(self)->list:
        return self.cursor.groups()
    
    def destroy(self):
        if self.is_open():
            print("Close connection...")
            self.close()
        
        print(f"Destroy << {self.hdf5_name}>>... ")
        os.remove(f"{self.hdf5_name}")
        print("Finish")
            
            

In [86]:
a = MyHDF5('new.h5')

<class 'pandas.io.pytables.HDFStore'>
File path: new.h5.h5
/test1                     frame        (shape->[20,4])
/test1/20210923            frame        (shape->[20,4])


In [87]:
a['test1']=employe

In [88]:
# t_hdf.destroy()

In [92]:
a.keys

['/test1', '/test1/20210923_114247']

In [13]:
len(t_hdf['employe'])

20

In [14]:
t_hdf.append_df('employe',employe)

In [15]:
len(t_hdf['employe'])

40

In [16]:
t_hdf.close()

Close success
